In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
train=pd.read_csv("/kaggle/input/home-data-for-ml-course/train.csv")
train['HouseAge']=train.YrSold-train.YearBuilt
train['AvgQuality']=(train.OverallQual+train.OverallCond)/2
train['TotalSF']=train.TotalBsmtSF+train['1stFlrSF']+train['2ndFlrSF']



In [3]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,HouseAge,AvgQuality,TotalSF
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,2,2008,WD,Normal,208500,5,6.0,2566
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,NaN,0,5,2007,WD,Normal,181500,31,7.0,2524
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,9,2008,WD,Normal,223500,7,6.0,2706
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,2,2006,WD,Abnorml,140000,91,6.0,2473
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,NaN,0,12,2008,WD,Normal,250000,8,6.5,3343


In [4]:
sub=pd.read_csv("/kaggle/input/home-data-for-ml-course/sample_submission.csv")


In [5]:
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder,FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split,cross_val_predict
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [6]:
y=train.SalePrice
X=train.drop(columns=['SalePrice'])



categorical=train.select_dtypes(include=['object'])
numeric=train.select_dtypes(include=['number']).drop(columns=['SalePrice'])
numeric_column=numeric.columns
categorical_column=categorical.columns
# skew=train[numeric_columns].apply(lambda x: x.skew() ).sort_values(ascending=False)
# skew_column=skew[skew>0.75].index
# numeric=train.select_dtypes(include=['number']).drop(columns=['SalePrice']).drop(columns=skew_column)
# numeric_column=numeric.columns




# 

# numeric_column
# categorical_column

In [7]:
categorical_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(handle_unknown='ignore'))
])
# skewed_transformer=Pipeline(steps=[
#     ('imputer',SimpleImputer(strategy='mean')),
#     ('log',FunctionTransformer(np.log1p,validate=True))
# ])
numeric_transformer=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='mean')),
    ('scaler',StandardScaler())
])

In [8]:
preprocessor=ColumnTransformer(transformers=[
    # ('skew',skewed_transformer,skew_column),
    ('num',numeric_transformer,numeric_column),
    ('cat',categorical_transformer,categorical_column)
    
])
pipeline=Pipeline(steps=[
    ('preprocessor',preprocessor),
    ('model',XGBRegressor())
])


In [9]:
pipeline.set_params(
    model__random_state=42,
    model__n_jobs=-1,
    model__n_estimators=1000,
    model__max_depth=5,
    model__subsample=0.9,
    model__colsample_bytree=0.7,
    model__reg_alpha=1,
    model__reg_lambda=0.1,
    model__learning_rate=0.05
)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Id', 'MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF'...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=5, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=-1,
                              num_parallel_tree=None, random_state=42, ...))])

In [10]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)
pipeline.fit(X_train,y_train)
pred=pipeline.predict(X_test)
print("error",mean_absolute_error(y_test,pred))
# cv=cross_val_predict(pipeline,X,y,cv=5)
# print("error",mean_absolute_error(y,cv))

error 14990.788131421234


In [11]:
test=pd.read_csv("/kaggle/input/home-data-for-ml-course/test.csv")
test['HouseAge']=test.YrSold-test.YearBuilt
test['AvgQuality']=(test.OverallQual+test.OverallCond)/2
test['TotalSF']=test.TotalBsmtSF+test['1stFlrSF']+test['2ndFlrSF']






In [12]:
test_pred=pipeline.predict(test)

output=pd.DataFrame({"Id": test.Id,"SalePrice": test_pred})
output.to_csv("submissions.csv",index=False)
output


,Id,SalePrice
0,1461,135618.656250
1,1462,167504.203125
2,1463,190275.843750
3,1464,193214.000000
4,1465,203811.140625
...,...,...
1454,2915,83409.117188
1455,2916,81759.984375
1456,2917,166171.218750
1457,2918,121397.046875


In [13]:
# param={
#     'model__n_estimators':[100,500,1000],
#     'model__max_depth':[3,5,7],
#     'model__learning_rate':[0.01,0.05,0.1],
#     'model__reg_alpha':[0,0.1,1],
#     'model__reg_lambda':[0,0.1,1],
#     'model__colsample_bytree':[0.5,0.7,0.9],
#     'model__subsample':[0.5,0.7,0.9]
    
# }
# ran=RandomizedSearchCV(
#     estimator=pipeline,
#     param_distributions=param,
#     cv=5,
#     scoring='neg_mean_absolute_error',
#     verbose=2,
#     n_jobs=-1,
#     random_state=42
# )
# ran.fit(X_train,y_train)
# best_model=ran.best_estimator_
# pred=best_model.predict(X_test)
# print("best params",ran.best_params_)
# print("error",mean_absolute_error(y_test,pred))


In [14]:


# model=XGBRegressor(
#     n_estimators=1000,
#     learning_rate=0.01,
#     max_depth=5,
#     reg_alpha=0,
#     reg_lambda=0.1,
#     subsample=0.7,
#     colsample_bytree=0.8,
#     random_state=0,
#     n_jobs=-1
    
# )
# model.fit(X_train,y_train)
# predict=model.predict(X_test)
# print("error",mean_absolute_error(predict,y_test))
# model=XGBRegressor(n_jobs=-1,random_state=0)
# param_grid={
#     'n_estimators':[100,500,1000],
#     'learning_rate':[0.01,0.05,0.1],
#     'max_depth':[3,5,7],
#     'reg_alpha':[0,0.1,1],
#     'reg_lambda':[0,0.1,1],
#     'subsample':[0.7,0.8,1],
#     'colsample_bytree':[0.7,0.8,1]
# }
# grid_search=GridSearchCV(
#     estimator=model,
#     param_grid=param_grid,
#     cv=3,
#     scoring='neg_mean_absolute_error',
#     verbose=1,
#     n_jobs=-1
# )
# grid_search.fit(X_train,y_train)
# best_model=grid_search.best_estimator_
# pred=best_model.predict(X_test)
# print("Best Parameters:", grid_search.best_params_)
# print("error", mean_absolute_error(y_test, pred))
# model.fit(X_train,y_train)
# predict=model.predict(X_test)
# print("mean squared error: ",mean_absolute_error(y_test,predict))




In [15]:
# test_preds=best_model.predict(X_test_main)
# output=pd.DataFrame({"ID":test_data.index,"Sale Price": test_preds})
# output.to_csv("submissions.csv",index=False)
